In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import math
import joblib

from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_csv('train.csv')
df = pd.concat([df, pd.get_dummies(df['Faculty'])], axis='columns')

models = {}

c = 0
for course in df["Course"].unique():
    df_i = df[df["Course"] == course]
    del df_i["Course"]
    
    df_i['Year'] = df_i['Timestep'].apply(lambda x: int(x[2:6]))
    
    x_i = df_i.drop(['Id', 'Timestep', 'Faculty', 'Expected'], axis = 'columns')
    
    cols = x_i.columns.tolist()
    
    y_i = df_i['Expected']
     
    model = CatBoostRegressor(
                                learning_rate=0.75,
                                depth = 4,
                                n_estimators = 100
    )

    model.fit(x_i, y_i,
              verbose = False,
              plot = False)
    
    models[course] = model
    c+=1
    print(c)

In [ ]:
cols = cols[-1:] + cols[:-1]
cols

In [ ]:
df = pd.read_csv('test.csv')
df = pd.concat([df, pd.get_dummies(df['Faculty'])], axis='columns')
df['Year'] = df['Timestep'].apply(lambda x: int(x[2:6]))

sol = []

for i, _ in df.iterrows():
    df_i = df[i:i+1]
    
    course = df_i["Course"][i]
    del df_i["Course"]

    x_i = df_i.drop(['Id', 'Timestep', 'Faculty'], axis = 'columns')
    x_i = x_i[cols]

    
    y_pred = models[course].predict(x_i)
    sol.append(y_pred[0])
    print(i)

In [ ]:
ans = pd.DataFrame({"Id": [i for i in range(10000)], "Predicted": sol})


In [ ]:
ans.to_csv("sub_last_one.csv", index=False)